In [1]:
import os
import pandas as pd
import pyreadstat

In [5]:
root_path = r"D:\临床数据\NHANES数据清洗\NHANES 2000-2020\2013-2014"

dfs = []

for subdir, dirs, files in os.walk(root_path):
    for file in files:
        if file.endswith(".XPT"):
            file_path = os.path.join(subdir, file)
            try:
                df, _ = pyreadstat.read_xport(file_path, encoding='latin1')
                if 'SEQN' in df.columns:
                    dfs.append(df)
                else:
                    print(f"File {file_path} does not contain 'SEQN' column and will be skipped.")
            except UnicodeDecodeError as e:
                print(f"Unicode error reading {file_path}: {e}")
            except Exception as e:
                print(f"Error reading {file_path}: {e}")

if dfs:
    merged_df = dfs.pop(0)
    for i, df in enumerate(dfs):
        common_cols = merged_df.columns.intersection(df.columns).tolist()
        common_cols.remove('SEQN')
        df = df.rename(columns={col: f"{col}_{i}" for col in common_cols})
        merged_df = pd.merge(merged_df, df, on="SEQN", how="outer")

    output_file = r"D:\临床数据\NHANES数据清洗\2013-2014_merged_file.csv"
    merged_df.to_csv(output_file, index=False)
    print(f"Merged file saved as {output_file}")
else:
    print("No valid dataframes to merge.")

File D:\临床数据\NHANES数据清洗\NHANES 2000-2020\2013-2014\2013-2014 Laboratory Data - Continuous NHANES\Brominated Flame Retardants (BFRs) - Pooled Samples.XPT does not contain 'SEQN' column and will be skipped.
File D:\临床数据\NHANES数据清洗\NHANES 2000-2020\2013-2014\2013-2014 Laboratory Data - Continuous NHANES\Non-dioxin-like Polychlorinated Biphenyls & Mono-ortho-substituted Polychlorinated Biphenyls - Serum - Pooled Samples.XPT does not contain 'SEQN' column and will be skipped.
File D:\临床数据\NHANES数据清洗\NHANES 2000-2020\2013-2014\2013-2014 Laboratory Data - Continuous NHANES\Pesticides - Organochlorine Pesticides - Serum - Pooled Samples.XPT does not contain 'SEQN' column and will be skipped.
Merged file saved as D:\临床数据\NHANES数据清洗\2013-2014_merged_file.csv


In [6]:
directory_path = 'D:\\临床数据\\NHANES数据清洗\\2007-2014_merged_file'

df_list = []

for filename in os.listdir(directory_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(directory_path, filename)
        df = pd.read_csv(file_path, low_memory=False)
        df_list.append(df)

merged_df = pd.concat(df_list, ignore_index=True)

print(merged_df.info())

output_file_path = 'D:\\临床数据\\NHANES数据清洗\\merged_data.csv'
merged_df.to_csv(output_file_path, index=False)

print(f"数据已成功保存到 {output_file_path}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40643 entries, 0 to 40642
Columns: 1959 entries, SEQN to SMQ852U
dtypes: float64(1957), object(2)
memory usage: 607.4+ MB
None
数据已成功保存到 D:\临床数据\NHANES数据清洗\merged_data.csv


In [7]:
file_path = 'D:\\临床数据\\NHANES数据清洗\\merged_data.csv'

df = pd.read_csv(file_path, low_memory=False)

missing_ratio = df.isnull().mean()

print(missing_ratio)

SEQN        0.000000
SDDSRVYR    0.000000
RIDSTATR    0.000000
RIDEXMON    0.035701
RIAGENDR    0.000000
              ...   
SMQ665C     0.999779
SMQ665D     0.999951
SMQ848      0.985656
SMQ852Q     0.985729
SMQ852U     0.985803
Length: 1959, dtype: float64


In [9]:
columns_to_drop = missing_ratio[missing_ratio > 0.5].index
df1 = df.drop(columns=columns_to_drop)

In [11]:
df2 = df1.drop_duplicates()

In [14]:
output_file_path = 'D:\\临床数据\\NHANES数据清洗\\删除缺失值列.csv'

df2.to_csv(output_file_path, index=False)

In [30]:
folder_path = "D:/临床数据/NHANES数据清洗/死亡结局数据"

column_specs = [
    (0, 6),
    (14, 15), 
    (15, 16), 
    (16, 19), 
    (19, 20),
    (20, 21), 
    (42, 45), 
    (45, 48)  
]

column_names = [
    "SEQN", "eligstat", "mortstat", "ucod_leading", 
    "diabetes", "hyperten", "permth_int", "permth_exm"
]

all_data = []

for file_name in os.listdir(folder_path):
    if file_name.endswith(".dat"):
        file_path = os.path.join(folder_path, file_name)
        
        data = pd.read_fwf(
            file_path, 
            colspecs=column_specs, 
            names=column_names, 
            na_values=["", "."]
        )
        
        all_data.append(data)

merged_data = pd.concat(all_data, ignore_index=True)

In [32]:
file_path = 'D:\\临床数据\\NHANES数据清洗\\删除缺失值列.csv'

df = pd.read_csv(file_path, low_memory=False)

In [34]:
merged_df = pd.merge(merged_data, df, on="SEQN", how="outer")

In [36]:
output_file = r"D:\临床数据\NHANES数据清洗\合并死亡数据.csv"
merged_df.to_csv(output_file, index=False)

In [39]:
file_path = 'D:\\临床数据\\NHANES数据清洗\\AGE=20+.csv'

df = pd.read_csv(file_path, low_memory=False)

In [43]:
missing_ratio = df.iloc[:, 6:62].isna().mean(axis=1)

df_cleaned = df[missing_ratio <= 0.5].reset_index(drop=True)

In [45]:
missing_ratio_cols = df_cleaned.iloc[:, 6:62].isna().mean()

In [51]:
output_file = r"D:\临床数据\NHANES数据清洗\删除缺失值行.csv"
df_cleaned.to_csv(output_file, index=False)